In [ ]:
# Importing for model 1
from transformers import AutoModelForSequenceClassification,RobertaTokenizer, Trainer, TrainingArguments
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
import numpy as np
import os

# Dataset class
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.label = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
       
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation = True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding =  'max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
   


        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'label': torch.tensor(self.label[index], dtype=torch.float)
        }

# %%

def encode_Lables(label):
    temp = np.zeros(5)
    temp[label-1] = 1
    return temp


    
def split_data(filename,samp_size):
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    path = parent_dir +"/model_data/" + filename
    #Sampling data
    if samp_size > 1:
        samp_size = 1
    if samp_size <=0:
      samp_size = .1

    df = pd.read_csv(path)  
    df = df.sample(n=int(len(df) * samp_size))
    df.reset_index(inplace=True)
    df = pd.read_csv(path)
    df = df.sample(n=int(len(df) * .50), replace=True)

    #Renaming for Model
    df.rename(columns={"Rating":"label"},inplace=True)
    df.rename(columns={"Review_text":"text"},inplace=True)
    df["label"] = df["label"].apply(encode_Lables)
    train_data,test_data = train_test_split(df,test_size=.20,random_state=64)
    train_data = train_data.reset_index(drop=True)
    test_data = test_data.reset_index(drop=True)
    max_length = int(df["text"].str.split().str.len().mean())
    return train_data,test_data,max_length

#hyperParmaters for roberta
samp_size = .1
lr = 2e-5
epochs = 5
train_batch_size = 8
test_batch_size = 8
w_decay = .001

#hyperParmaters for MetaModel
m_lr = 2e-5
m_epochs = 5
m_w_decay = .001

#Setting filename and creating datasets for both models
filename = "data_set_1.csv"
train,test,max_length= split_data(filename,samp_size)

# Creating Model and Tokenizer
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=5)
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Creating Datasets
training_set = SentimentData(train, tokenizer, max_len=max_length)
testing_set = SentimentData(test, tokenizer, max_len=max_length)



# Creating Metric function
def compute_metrics(pred):
    
    label = torch.argmax(torch.tensor(pred.label_ids),dim=1).numpy()
    preds = torch.argmax(torch.tensor(pred.predictions),dim=1).numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(label, preds, average='macro')
    acc = accuracy_score(label, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Setting Training paramaters
training_args = TrainingArguments(

    learning_rate=lr,
    fp16=True,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=test_batch_size,
    num_train_epochs=epochs,
    weight_decay=w_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_set,
    eval_dataset=testing_set,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training Model
trainer.train()

pred_Roberta = trainer.predict()


In [ ]:
import xgboost as xgb

#Create Tree
tree = XGBClassifier(n_estimators=5, max_depth=2, learning_rate=1, objective='binary:logistic')

#Train model
tree.fit(train["text"],train["label"])

pred_XGBoost = tree.predict(test["label"])

In [ ]:
class MetaModel(torch.nn.Module):
    def __init__(self, n_layers, hidden_size):
        super(MetaModel, self).__init__()

        self.hidden = nn.ModuleList()
        self.hidden.append(nn.Linear(5, hidden_size))
        for _ in range(n_layers - 1):
            self.hidden.append(nn.Linear(hidden_size, hidden_size))

        self.output = nn.Linear(hidden_size, 5)

        self.relu = torch.nn.Relu()

        self.softMax = torch.nn.LogSoftmax(dim = -1)
    def foward(self,x):

        for layer in self.hidden:
            x = self.relu(layer(x))

        x = self.output(x)
        return x

In [ ]:
metamodel = MetaModel(2,10)

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(metamodel.paramaters(),lr=m_lr,weight_decay=m_w_decay)



    

In [ ]:
#training Loop
for range in m_epochs: